In [3]:
import fitz  # PyMuPDF
from pathlib import Path

pdf_path = Path("Modelo_PDF (1).pdf")

# Abrir o PDF
pdf_document = fitz.open(pdf_path)

# Verificar se o PDF foi carregado corretamente
if pdf_document.page_count > 0:
    txt_pag = {}
    for i in range(pdf_document.page_count):
        page = pdf_document.load_page(i)
        texto = page.get_text("text")
        txt_pag[i] = texto
    for pagina, conteudo in txt_pag.items():
        print(f"Pagina {pagina + 1}:")
        print(conteudo)
        print("\n" + "-" * 50 + "\n")
else:
    print("O PDF inserido está vazio")


AttributeError: 'Document' object has no attribute 'get_document'